In [60]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [61]:
data = pd.read_csv('Datasets/sales_train.csv')

In [62]:
calendar = pd.read_csv('Datasets/calendar.csv')

In [63]:
calendar.loc[calendar['date']=='2021-12-10']

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
1091,2021-12-10,NaN,0,0,0,0,Prague_2
6220,2021-12-10,NaN,0,0,0,0,Munich_1
9518,2021-12-10,NaN,0,0,0,0,Prague_3
15465,2021-12-10,NaN,0,0,0,0,Brno_1
15469,2021-12-10,NaN,0,0,0,0,Budapest_1
18068,2021-12-10,NaN,0,0,0,0,Prague_1
18978,2021-12-10,NaN,0,0,0,0,Frankfurt_1


In [64]:
inventory = pd.read_csv('Datasets/inventory.csv')


In [65]:
inventory.loc[inventory['unique_id']==885]

,unique_id,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse
4029,885,460,Tomato_19,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_40,Fruit and vegetable_L4_1,Munich_1


In [66]:
inventory['name'].head()

0    Pastry_196
1       Herb_19
2        Beet_2
3    Chicken_13
4     Chicory_1
Name: name, dtype: object

In [67]:
test_weight = pd.read_csv(r'Datasets/test_weights.csv')


Lets us apply feature engineering in sales-train

In [68]:
calendar.isnull().sum()

date                          0
holiday_name              22086
holiday                       0
shops_closed                  0
winter_school_holidays        0
school_holidays               0
warehouse                     0
dtype: int64

In [69]:
data.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,0.0,0.15312,0.0,0.0
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,0.0,0.15312,0.0,0.0
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,0.0,0.15649,0.0,0.0


In [70]:
data['warehouse'].unique()

array(['Budapest_1', 'Prague_2', 'Brno_1', 'Prague_1', 'Prague_3',
       'Munich_1', 'Frankfurt_1'], dtype=object)

In [71]:
data.isnull().sum()

unique_id           0
date                0
warehouse           0
total_orders       52
sales              52
sell_price_main     0
availability        0
type_0_discount     0
type_1_discount     0
type_2_discount     0
type_3_discount     0
type_4_discount     0
type_5_discount     0
type_6_discount     0
dtype: int64

In [72]:
data['total_orders'].isnull().sum()

52

Visualization in sales_train

In [73]:
"""#Box plot
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
sns.boxplot(x=data['sales'])
plt.title("Boxplot of Sales (Outlier Detection)")
plt.show()
"""

'#Box plot\nimport seaborn as sns\nimport matplotlib.pyplot as plt\n\nplt.figure(figsize=(10,5))\nsns.boxplot(x=data[\'sales\'])\nplt.title("Boxplot of Sales (Outlier Detection)")\nplt.show()\n'

In [74]:
"""import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.heatmap(data.corr(), annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()
"""

'import seaborn as sns\nimport matplotlib.pyplot as plt\n\nplt.figure(figsize=(10,6))\nsns.heatmap(data.corr(), annot=True, cmap="coolwarm", linewidths=0.5)\nplt.title("Feature Correlation Heatmap")\nplt.show()\n'

In [75]:
"""plt.figure(figsize=(10,5))
sns.histplot(data['sales'], bins=100, kde=True)
plt.title("Sales Distribution")
plt.xlabel("Sales Count")
plt.ylabel("Frequency")
plt.show()
"""

'plt.figure(figsize=(10,5))\nsns.histplot(data[\'sales\'], bins=100, kde=True)\nplt.title("Sales Distribution")\nplt.xlabel("Sales Count")\nplt.ylabel("Frequency")\nplt.show()\n'

In [76]:
skewness = data['sales'].skew()
skewness

22.36323642023012

In [77]:
# Define the clipping range
lower_bound = data['sales'].quantile(0.05)
upper_bound = data['sales'].quantile(0.95)

# Clip the sales data
data['sales'] = data['sales'].clip(lower=lower_bound, upper=upper_bound)

# Check the skewness after clipping
data['sales'] = data['sales'].skew()


"""from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

data['sales'] = pd.DataFrame(imputer.fit_transform(data[['sales']]))"""

"from sklearn.impute import SimpleImputer\n\nimputer = SimpleImputer(strategy='median')\n\ndata['sales'] = pd.DataFrame(imputer.fit_transform(data[['sales']]))"

In [78]:
data['sales'].isnull().sum()

0

In [79]:
"""plt.figure(figsize=(10,5))
sns.histplot(data['sales'], bins=100, kde=True)
plt.title("Sales Distribution")
plt.xlabel("Sales Count")
plt.ylabel("Frequency")
plt.show()
"""

'plt.figure(figsize=(10,5))\nsns.histplot(data[\'sales\'], bins=100, kde=True)\nplt.title("Sales Distribution")\nplt.xlabel("Sales Count")\nplt.ylabel("Frequency")\nplt.show()\n'

In [80]:
"""plt.figure(figsize=(12,5))
data.groupby('date')['sales'].sum().plot(kind='line')
plt.title("Sales Over Time")
plt.xlabel("Date")
plt.ylabel("Total Sales")
plt.show()
"""

'plt.figure(figsize=(12,5))\ndata.groupby(\'date\')[\'sales\'].sum().plot(kind=\'line\')\nplt.title("Sales Over Time")\nplt.xlabel("Date")\nplt.ylabel("Total Sales")\nplt.show()\n'

In [81]:
"""plt.figure(figsize=(12,5))
sns.boxplot(x='unique_id', y='sales', data=data)
plt.title("Sales Distribution per Store")
plt.xticks(rotation=90)
plt.show()
"""

'plt.figure(figsize=(12,5))\nsns.boxplot(x=\'unique_id\', y=\'sales\', data=data)\nplt.title("Sales Distribution per Store")\nplt.xticks(rotation=90)\nplt.show()\n'

In [82]:
data.isnull().sum()

unique_id           0
date                0
warehouse           0
total_orders       52
sales               0
sell_price_main     0
availability        0
type_0_discount     0
type_1_discount     0
type_2_discount     0
type_3_discount     0
type_4_discount     0
type_5_discount     0
type_6_discount     0
dtype: int64

In [83]:
data['total_orders']

0           6436.0
1           4663.0
2           6507.0
3           5463.0
4           5997.0
            ...   
4007414     9988.0
4007415     8518.0
4007416    10424.0
4007417    10342.0
4007418     9613.0
Name: total_orders, Length: 4007419, dtype: float64

In [84]:
"""import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
sns.barplot(x=data['unique_id'], y=data['total_orders'], estimator=sum, ci=None)
plt.xticks(rotation=90)
plt.title("Total Orders by Store")
plt.xlabel("Store ID")
plt.ylabel("Total Orders")
plt.show()"""


'import seaborn as sns\nimport matplotlib.pyplot as plt\n\nplt.figure(figsize=(12,6))\nsns.barplot(x=data[\'unique_id\'], y=data[\'total_orders\'], estimator=sum, ci=None)\nplt.xticks(rotation=90)\nplt.title("Total Orders by Store")\nplt.xlabel("Store ID")\nplt.ylabel("Total Orders")\nplt.show()'

Merging all the other datas with the main sales_train

In [85]:
calendar.head()

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
0,2022-03-16,NaN,0,0,0,0,Frankfurt_1
1,2020-03-22,NaN,0,0,0,0,Frankfurt_1
2,2018-02-07,NaN,0,0,0,0,Frankfurt_1
3,2018-08-10,NaN,0,0,0,0,Frankfurt_1
4,2017-10-26,NaN,0,0,0,0,Prague_2


In [86]:
inventory.head()

,unique_id,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse
0,5255,2583,Pastry_196,Bakery,Bakery_L2_14,Bakery_L3_26,Bakery_L4_1,Prague_3
1,4948,2426,Herb_19,Fruit and vegetable,Fruit and vegetable_L2_30,Fruit and vegetable_L3_86,Fruit and vegetable_L4_1,Prague_3
2,2146,1079,Beet_2,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_65,Fruit and vegetable_L4_34,Prague_1
3,501,260,Chicken_13,Meat and fish,Meat and fish_L2_13,Meat and fish_L3_27,Meat and fish_L4_5,Prague_1
4,4461,2197,Chicory_1,Fruit and vegetable,Fruit and vegetable_L2_17,Fruit and vegetable_L3_33,Fruit and vegetable_L4_1,Frankfurt_1


In [87]:
data.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
0,4845,2024-03-10,Budapest_1,6436.0,1.929869,646.26,1.00,0.00000,0.0,0.0,0.0,0.15312,0.0,0.0
1,4845,2021-05-25,Budapest_1,4663.0,1.929869,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
2,4845,2021-12-20,Budapest_1,6507.0,1.929869,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
3,4845,2023-04-29,Budapest_1,5463.0,1.929869,646.26,0.96,0.20024,0.0,0.0,0.0,0.15312,0.0,0.0
4,4845,2022-04-01,Budapest_1,5997.0,1.929869,486.41,1.00,0.00000,0.0,0.0,0.0,0.15649,0.0,0.0


In [88]:
data = data.merge(inventory[['unique_id', 'product_unique_id', 'name',
                                'L1_category_name_en', 'L2_category_name_en',
                                'L3_category_name_en', 'L4_category_name_en']],
                on='unique_id', how='left')

In [89]:
data = data.merge(calendar[['date', 'warehouse', 'holiday', 'holiday_name',
                                'shops_closed', 'winter_school_holidays', 'school_holidays']],
                on=['date', 'warehouse'], how='left')


In [90]:
data.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,winter_school_holidays,school_holidays
0,4845,2024-03-10,Budapest_1,6436.0,1.929869,646.26,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0
1,4845,2021-05-25,Budapest_1,4663.0,1.929869,455.96,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0
2,4845,2021-12-20,Budapest_1,6507.0,1.929869,455.96,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0
3,4845,2023-04-29,Budapest_1,5463.0,1.929869,646.26,0.96,0.20024,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0
4,4845,2022-04-01,Budapest_1,5997.0,1.929869,486.41,1.00,0.00000,0.0,0.0,...,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0


In [91]:
df = data[data['total_orders'].isna()]
df.tail()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,winter_school_holidays,school_holidays
3940655,2809,2021-06-13,Munich_1,NaN,1.929869,6.05,1.0,0.09783,0.0,0.0,...,Nectarine_9,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_57,Fruit and vegetable_L4_1,0,NaN,0,0,0
3940665,2809,2021-06-27,Munich_1,NaN,1.929869,5.32,1.0,0.00000,0.0,0.0,...,Nectarine_9,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_57,Fruit and vegetable_L4_1,0,NaN,0,0,0
3940702,2809,2021-06-26,Munich_1,NaN,1.929869,5.32,1.0,0.00000,0.0,0.0,...,Nectarine_9,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_57,Fruit and vegetable_L4_1,0,NaN,0,0,0
4002178,794,2021-12-10,Frankfurt_1,NaN,1.929869,1.56,0.3,0.00000,0.0,0.0,...,Carrot_7,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_33,Fruit and vegetable_L4_1,0,NaN,0,0,0
4002183,794,2021-12-09,Frankfurt_1,NaN,1.929869,1.56,1.0,0.00000,0.0,0.0,...,Carrot_7,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_33,Fruit and vegetable_L4_1,0,NaN,0,0,0


In [ ]:
data.isnull().sum()

In [274]:
data = data.merge(test_weight[['unique_id','weight']],on='unique_id', how='left')

In [275]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 26 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   unique_id               int64  
 1   date                    object 
 2   warehouse               object 
 3   total_orders            float64
 4   sales                   float64
 5   sell_price_main         float64
 6   availability            float64
 7   type_0_discount         float64
 8   type_1_discount         float64
 9   type_2_discount         float64
 10  type_3_discount         float64
 11  type_4_discount         float64
 12  type_5_discount         float64
 13  type_6_discount         float64
 14  product_unique_id       int64  
 15  name                    object 
 16  L1_category_name_en     object 
 17  L2_category_name_en     object 
 18  L3_category_name_en     object 
 19  L4_category_name_en     object 
 20  holiday                 int64  
 21  holiday_name            object 

In [276]:
data.to_csv('Final_Raw_data/Final_data.csv',index=True)

Working on the final data

In [277]:
final_data = pd.read_csv('Final_Raw_data/Final_data.csv')

In [278]:
final_data.head()

,Unnamed: 0,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,...,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,winter_school_holidays,school_holidays,weight
0,0,4845,2024-03-10,Budapest_1,6436.0,1.929869,646.26,1.00,0.00000,0.0,...,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0,1.925596
1,1,4845,2021-05-25,Budapest_1,4663.0,1.929869,455.96,1.00,0.00000,0.0,...,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0,1.925596
2,2,4845,2021-12-20,Budapest_1,6507.0,1.929869,455.96,1.00,0.00000,0.0,...,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0,1.925596
3,3,4845,2023-04-29,Budapest_1,5463.0,1.929869,646.26,0.96,0.20024,0.0,...,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0,1.925596
4,4,4845,2022-04-01,Budapest_1,5997.0,1.929869,486.41,1.00,0.00000,0.0,...,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,0,NaN,0,0,0,1.925596


In [279]:
final_data.isnull().sum()

Unnamed: 0                      0
unique_id                       0
date                            0
warehouse                       0
total_orders                   52
sales                           0
sell_price_main                 0
availability                    0
type_0_discount                 0
type_1_discount                 0
type_2_discount                 0
type_3_discount                 0
type_4_discount                 0
type_5_discount                 0
type_6_discount                 0
product_unique_id               0
name                            0
L1_category_name_en             0
L2_category_name_en             0
L3_category_name_en             0
L4_category_name_en             0
holiday                         0
holiday_name              3844119
shops_closed                    0
winter_school_holidays          0
school_holidays                 0
weight                          0
dtype: int64

In [280]:
#Handling the nulll values
final_data['total_orders'] = final_data['total_orders'].fillna(0)


In [281]:
final_data['holiday_name'].unique()

array([nan, 'Christmas Eve', 'State Foundation Day',
       'Day of National Unity',
       'Memorial Day for the Victims of the Holocaust',
       'Memorial Day for the Martyrs of Arad', 'Easter Monday',
       'Labour Day', 'New Years Day', 'Whit sunday', '1st Christmas Day',
       'National Defense Day', 'Hungary National Day Holiday',
       '1848 Revolution Memorial Day (Extra holiday)',
       'Memorial day of the 1956 Revolution', 'International womens day',
       'Independent Hungary Day', 'Whit monday',
       'Memorial Day of the Republic', "All Saints' Day Holiday",
       'All Saints Day', '2nd Christmas Day', 'Good Friday',
       'Memorial Day for the Victims of the Communist Dictatorships',
       'Den vzniku samostatneho ceskoslovenskeho statu',
       'Cyrila a Metodej', 'Den boje za svobodu a demokracii',
       'Den osvobozeni', 'Den ceske statnosti', 'Jan Hus',
       'Peace Festival in Augsburg', 'Ascension day', 'Epiphany',
       'Corpus Christi', 'Assumption o

In [282]:
#Handling the missing values of total_orders
final_data['holiday_name'] = final_data['holiday_name'].fillna('No Holiday')


In [283]:
final_data.isnull().sum()

Unnamed: 0                0
unique_id                 0
date                      0
warehouse                 0
total_orders              0
sales                     0
sell_price_main           0
availability              0
type_0_discount           0
type_1_discount           0
type_2_discount           0
type_3_discount           0
type_4_discount           0
type_5_discount           0
type_6_discount           0
product_unique_id         0
name                      0
L1_category_name_en       0
L2_category_name_en       0
L3_category_name_en       0
L4_category_name_en       0
holiday                   0
holiday_name              0
shops_closed              0
winter_school_holidays    0
school_holidays           0
weight                    0
dtype: int64

In [284]:
#Converting the types discount into the min and max discount
final_data['min_discount'] = final_data[['type_0_discount', 'type_1_discount', 'type_2_discount',
                                          'type_3_discount', 'type_4_discount', 'type_5_discount',
                                          'type_6_discount']].min(axis=1)


final_data['max_discount'] = final_data[['type_0_discount', 'type_1_discount', 'type_2_discount',
                                  'type_3_discount', 'type_4_discount', 'type_5_discount',
                                  'type_6_discount']].max(axis=1)


In [285]:
final_data.drop(columns=['type_0_discount', 'type_1_discount', 'type_2_discount',
                                  'type_3_discount', 'type_4_discount', 'type_5_discount',
                                  'type_6_discount'],inplace=True)

In [286]:
final_data.head()

,Unnamed: 0,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,product_unique_id,name,...,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,winter_school_holidays,school_holidays,weight,min_discount,max_discount
0,0,4845,2024-03-10,Budapest_1,6436.0,1.929869,646.26,1.00,2375,Croissant_35,...,Bakery_L3_83,Bakery_L4_1,0,No Holiday,0,0,0,1.925596,0.0,0.15312
1,1,4845,2021-05-25,Budapest_1,4663.0,1.929869,455.96,1.00,2375,Croissant_35,...,Bakery_L3_83,Bakery_L4_1,0,No Holiday,0,0,0,1.925596,0.0,0.15025
2,2,4845,2021-12-20,Budapest_1,6507.0,1.929869,455.96,1.00,2375,Croissant_35,...,Bakery_L3_83,Bakery_L4_1,0,No Holiday,0,0,0,1.925596,0.0,0.15025
3,3,4845,2023-04-29,Budapest_1,5463.0,1.929869,646.26,0.96,2375,Croissant_35,...,Bakery_L3_83,Bakery_L4_1,0,No Holiday,0,0,0,1.925596,0.0,0.20024
4,4,4845,2022-04-01,Budapest_1,5997.0,1.929869,486.41,1.00,2375,Croissant_35,...,Bakery_L3_83,Bakery_L4_1,0,No Holiday,0,0,0,1.925596,0.0,0.15649


In [287]:
#date handling
final_data['date'] = pd.to_datetime(final_data['date'])

In [288]:
# Extract year, month, day, and weekday
final_data['year'] = final_data['date'].dt.year
final_data['month'] = final_data['date'].dt.month
final_data['day'] = final_data['date'].dt.day
final_data['weekday'] = final_data['date'].dt.weekday  # 0=Monday, 6=Sunday
final_data['day_of_week'] = final_data['date'].dt.day_name()  # Weekday name


In [289]:
final_data.drop(columns='date',inplace=True)

In [290]:
final_data.head()

,Unnamed: 0,unique_id,warehouse,total_orders,sales,sell_price_main,availability,product_unique_id,name,L1_category_name_en,...,winter_school_holidays,school_holidays,weight,min_discount,max_discount,year,month,day,weekday,day_of_week
0,0,4845,Budapest_1,6436.0,1.929869,646.26,1.00,2375,Croissant_35,Bakery,...,0,0,1.925596,0.0,0.15312,2024,3,10,6,Sunday
1,1,4845,Budapest_1,4663.0,1.929869,455.96,1.00,2375,Croissant_35,Bakery,...,0,0,1.925596,0.0,0.15025,2021,5,25,1,Tuesday
2,2,4845,Budapest_1,6507.0,1.929869,455.96,1.00,2375,Croissant_35,Bakery,...,0,0,1.925596,0.0,0.15025,2021,12,20,0,Monday
3,3,4845,Budapest_1,5463.0,1.929869,646.26,0.96,2375,Croissant_35,Bakery,...,0,0,1.925596,0.0,0.20024,2023,4,29,5,Saturday
4,4,4845,Budapest_1,5997.0,1.929869,486.41,1.00,2375,Croissant_35,Bakery,...,0,0,1.925596,0.0,0.15649,2022,4,1,4,Friday


In [291]:
final_data.drop(columns=['Unnamed: 0','unique_id','winter_school_holidays','school_holidays'],inplace=True)

In [292]:
final_data.shape

(4007419, 22)

In [293]:
# Fetch categorical columns
categorical_columns = final_data.select_dtypes(include=['object', 'category']).columns
numerical_columns = final_data.select_dtypes(include=['int', 'category']).columns



# Display the categorical columns
print(categorical_columns)
print(numerical_columns)


Index(['warehouse', 'name', 'L1_category_name_en', 'L2_category_name_en',
       'L3_category_name_en', 'L4_category_name_en', 'holiday_name',
       'day_of_week'],
      dtype='object')
Index(['product_unique_id', 'holiday', 'shops_closed', 'year', 'month', 'day',
       'weekday'],
      dtype='object')


In [294]:
"""# Combine all category levels into one single string column
final_data['combined_category'] = final_data['L1_category_name_en'] + " > " + \
                                  final_data['L2_category_name_en'] + " > " + \
                                  final_data['L3_category_nam"e_en'] + " > " + \
                                  final_data['L4_category_name_en']"""


'# Combine all category levels into one single string column\nfinal_data[\'combined_category\'] = final_data[\'L1_category_name_en\'] + " > " +                                   final_data[\'L2_category_name_en\'] + " > " +                                   final_data[\'L3_category_nam"e_en\'] + " > " +                                   final_data[\'L4_category_name_en\']'

In [295]:
"final_data['combined_category']"

"final_data['combined_category']"

In [296]:
final_data.isnull().sum()

warehouse              0
total_orders           0
sales                  0
sell_price_main        0
availability           0
product_unique_id      0
name                   0
L1_category_name_en    0
L2_category_name_en    0
L3_category_name_en    0
L4_category_name_en    0
holiday                0
holiday_name           0
shops_closed           0
weight                 0
min_discount           0
max_discount           0
year                   0
month                  0
day                    0
weekday                0
day_of_week            0
dtype: int64

One hot encoding columns

1. warehouse
2. day_of_week

In [297]:
final_data['warehouse'].unique()

array(['Budapest_1', 'Prague_2', 'Brno_1', 'Prague_1', 'Prague_3',
       'Munich_1', 'Frankfurt_1'], dtype=object)

In [298]:
final_data['day_of_week'].unique()

array(['Sunday', 'Tuesday', 'Monday', 'Saturday', 'Friday', 'Thursday',
       'Wednesday'], dtype=object)

In [299]:
final_data['holiday_name'].unique()

array(['No Holiday', 'Christmas Eve', 'State Foundation Day',
       'Day of National Unity',
       'Memorial Day for the Victims of the Holocaust',
       'Memorial Day for the Martyrs of Arad', 'Easter Monday',
       'Labour Day', 'New Years Day', 'Whit sunday', '1st Christmas Day',
       'National Defense Day', 'Hungary National Day Holiday',
       '1848 Revolution Memorial Day (Extra holiday)',
       'Memorial day of the 1956 Revolution', 'International womens day',
       'Independent Hungary Day', 'Whit monday',
       'Memorial Day of the Republic', "All Saints' Day Holiday",
       'All Saints Day', '2nd Christmas Day', 'Good Friday',
       'Memorial Day for the Victims of the Communist Dictatorships',
       'Den vzniku samostatneho ceskoslovenskeho statu',
       'Cyrila a Metodej', 'Den boje za svobodu a demokracii',
       'Den osvobozeni', 'Den ceske statnosti', 'Jan Hus',
       'Peace Festival in Augsburg', 'Ascension day', 'Epiphany',
       'Corpus Christi', 'Ass

In [300]:
final_data.head()

,warehouse,total_orders,sales,sell_price_main,availability,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,...,holiday_name,shops_closed,weight,min_discount,max_discount,year,month,day,weekday,day_of_week
0,Budapest_1,6436.0,1.929869,646.26,1.00,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,...,No Holiday,0,1.925596,0.0,0.15312,2024,3,10,6,Sunday
1,Budapest_1,4663.0,1.929869,455.96,1.00,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,...,No Holiday,0,1.925596,0.0,0.15025,2021,5,25,1,Tuesday
2,Budapest_1,6507.0,1.929869,455.96,1.00,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,...,No Holiday,0,1.925596,0.0,0.15025,2021,12,20,0,Monday
3,Budapest_1,5463.0,1.929869,646.26,0.96,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,...,No Holiday,0,1.925596,0.0,0.20024,2023,4,29,5,Saturday
4,Budapest_1,5997.0,1.929869,486.41,1.00,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,...,No Holiday,0,1.925596,0.0,0.15649,2022,4,1,4,Friday


Performing one hot encoding and standard scaling

In [301]:
"""#categorical_columns = ['warehouse','day_of_week']

# Example: Numerical columns (you can modify these based on your dataset)
numerical_columns = ['sales','total_orders','sell_price_main','weight',]

# Step 2: One-Hot Encoding for Categorical Columns
# Perform One-Hot Encoding
#final_data = pd.get_dummies(final_data, columns=categorical_columns).astype(int)


# Step 3: Standard Scaling for Numerical Columns
scaler = StandardScaler()

# Apply StandardScaler to numerical columns
final_data[numerical_columns] = scaler.fit_transform(final_data[numerical_columns])"""


"#categorical_columns = ['warehouse','day_of_week']\n\n# Example: Numerical columns (you can modify these based on your dataset)\nnumerical_columns = ['sales','total_orders','sell_price_main','weight',]\n\n# Step 2: One-Hot Encoding for Categorical Columns\n# Perform One-Hot Encoding\n#final_data = pd.get_dummies(final_data, columns=categorical_columns).astype(int)\n\n\n# Step 3: Standard Scaling for Numerical Columns\nscaler = StandardScaler()\n\n# Apply StandardScaler to numerical columns\nfinal_data[numerical_columns] = scaler.fit_transform(final_data[numerical_columns])"

In [302]:
final_data.head()

,warehouse,total_orders,sales,sell_price_main,availability,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,...,holiday_name,shops_closed,weight,min_discount,max_discount,year,month,day,weekday,day_of_week
0,Budapest_1,6436.0,1.929869,646.26,1.00,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,...,No Holiday,0,1.925596,0.0,0.15312,2024,3,10,6,Sunday
1,Budapest_1,4663.0,1.929869,455.96,1.00,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,...,No Holiday,0,1.925596,0.0,0.15025,2021,5,25,1,Tuesday
2,Budapest_1,6507.0,1.929869,455.96,1.00,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,...,No Holiday,0,1.925596,0.0,0.15025,2021,12,20,0,Monday
3,Budapest_1,5463.0,1.929869,646.26,0.96,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,...,No Holiday,0,1.925596,0.0,0.20024,2023,4,29,5,Saturday
4,Budapest_1,5997.0,1.929869,486.41,1.00,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,...,No Holiday,0,1.925596,0.0,0.15649,2022,4,1,4,Friday


In [303]:
#Remove the extra spaces and symbols
column = ['warehouse','name','L1_category_name_en','L2_category_name_en','L3_category_name_en','L4_category_name_en','holiday_name',]


In [304]:
final_data['warehouse'] = final_data['warehouse'].str.replace('_', '', regex=False)
final_data['name'] = final_data['name'].str.replace('_', '', regex=False)
final_data['L1_category_name_en'] = final_data['L1_category_name_en'].str.replace('_', '', regex=False)
final_data['L2_category_name_en'] = final_data['L2_category_name_en'].str.replace('_', '', regex=False)
final_data['L3_category_name_en'] = final_data['L3_category_name_en'].str.replace('_', '', regex=False)
final_data['L4_category_name_en'] = final_data['L4_category_name_en'].str.replace('_', '', regex=False)
final_data['holiday_name'] = final_data['holiday_name'].str.replace('_', '', regex=False)


In [305]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Apply the label encoder to the 'warehouse' column
final_data['warehouse'] = label_encoder.fit_transform(final_data['warehouse'])
final_data['day_of_week'] = label_encoder.fit_transform(final_data['day_of_week'])


In [306]:
final_data.head()

,warehouse,total_orders,sales,sell_price_main,availability,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,...,holiday_name,shops_closed,weight,min_discount,max_discount,year,month,day,weekday,day_of_week
0,1,6436.0,1.929869,646.26,1.00,2375,Croissant35,Bakery,BakeryL218,BakeryL383,...,No Holiday,0,1.925596,0.0,0.15312,2024,3,10,6,3
1,1,4663.0,1.929869,455.96,1.00,2375,Croissant35,Bakery,BakeryL218,BakeryL383,...,No Holiday,0,1.925596,0.0,0.15025,2021,5,25,1,5
2,1,6507.0,1.929869,455.96,1.00,2375,Croissant35,Bakery,BakeryL218,BakeryL383,...,No Holiday,0,1.925596,0.0,0.15025,2021,12,20,0,1
3,1,5463.0,1.929869,646.26,0.96,2375,Croissant35,Bakery,BakeryL218,BakeryL383,...,No Holiday,0,1.925596,0.0,0.20024,2023,4,29,5,2
4,1,5997.0,1.929869,486.41,1.00,2375,Croissant35,Bakery,BakeryL218,BakeryL383,...,No Holiday,0,1.925596,0.0,0.15649,2022,4,1,4,0


In [307]:
final_data['name'] = final_data['name'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
final_data['L1_category_name_en'] = final_data['L1_category_name_en'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
final_data['L2_category_name_en'] = final_data['L2_category_name_en'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
final_data['L3_category_name_en'] = final_data['L3_category_name_en'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
final_data['L4_category_name_en'] = final_data['L4_category_name_en'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
final_data['holiday_name'] = final_data['holiday_name'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()


In [308]:
#performing encoding
final_data['name'] = label_encoder.fit_transform(final_data['name'])
final_data['L1_category_name_en'] = label_encoder.fit_transform(final_data['L1_category_name_en'])
final_data['L2_category_name_en'] = label_encoder.fit_transform(final_data['L2_category_name_en'])
final_data['L3_category_name_en'] = label_encoder.fit_transform(final_data['L3_category_name_en'])
final_data['L4_category_name_en'] = label_encoder.fit_transform(final_data['L4_category_name_en'])
final_data['holiday_name'] = label_encoder.fit_transform(final_data['holiday_name'])
final_data['day_of_week'] = label_encoder.fit_transform(final_data['day_of_week'])

In [309]:
final_data.head()

,warehouse,total_orders,sales,sell_price_main,availability,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,...,holiday_name,shops_closed,weight,min_discount,max_discount,year,month,day,weekday,day_of_week
0,1,6436.0,1.929869,646.26,1.00,2375,960,0,2,52,...,31,0,1.925596,0.0,0.15312,2024,3,10,6,3
1,1,4663.0,1.929869,455.96,1.00,2375,960,0,2,52,...,31,0,1.925596,0.0,0.15025,2021,5,25,1,5
2,1,6507.0,1.929869,455.96,1.00,2375,960,0,2,52,...,31,0,1.925596,0.0,0.15025,2021,12,20,0,1
3,1,5463.0,1.929869,646.26,0.96,2375,960,0,2,52,...,31,0,1.925596,0.0,0.20024,2023,4,29,5,2
4,1,5997.0,1.929869,486.41,1.00,2375,960,0,2,52,...,31,0,1.925596,0.0,0.15649,2022,4,1,4,0


In [310]:
#performing scaling
"""final_data['name'] = scaler.fit_transform(final_data[['name']])
final_data['L1_category_name_en'] = scaler.fit_transform(final_data[['L1_category_name_en']])
final_data['L2_category_name_en'] = scaler.fit_transform(final_data[['L2_category_name_en']])
final_data['L3_category_name_en'] = scaler.fit_transform(final_data[['L3_category_name_en']])
final_data['L4_category_name_en'] = scaler.fit_transform(final_data[['L4_category_name_en']])
final_data['holiday_name'] = scaler.fit_transform(final_data[['holiday_name']])"""


"final_data['name'] = scaler.fit_transform(final_data[['name']])\nfinal_data['L1_category_name_en'] = scaler.fit_transform(final_data[['L1_category_name_en']])\nfinal_data['L2_category_name_en'] = scaler.fit_transform(final_data[['L2_category_name_en']])\nfinal_data['L3_category_name_en'] = scaler.fit_transform(final_data[['L3_category_name_en']])\nfinal_data['L4_category_name_en'] = scaler.fit_transform(final_data[['L4_category_name_en']])\nfinal_data['holiday_name'] = scaler.fit_transform(final_data[['holiday_name']])"

In [311]:
final_data.tail()

,warehouse,total_orders,sales,sell_price_main,availability,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,...,holiday_name,shops_closed,weight,min_discount,max_discount,year,month,day,weekday,day_of_week
4007414,4,9988.0,1.929869,34.06,1.0,2422,1239,1,25,65,...,31,0,2.262646,0.0,0.0,2023,6,21,2,6
4007415,4,8518.0,1.929869,34.06,1.0,2422,1239,1,25,65,...,31,0,2.262646,0.0,0.0,2023,6,24,5,2
4007416,4,10424.0,1.929869,34.06,1.0,2422,1239,1,25,65,...,31,0,2.262646,0.0,0.0,2023,6,23,4,0
4007417,4,10342.0,1.929869,34.06,1.0,2422,1239,1,25,65,...,31,0,2.262646,0.0,0.0,2023,6,22,3,4
4007418,4,9613.0,1.929869,34.06,1.0,2422,1239,1,25,65,...,31,0,2.262646,0.0,0.0,2023,6,20,1,5


In [312]:
final_data['year'].unique()

array([2024, 2021, 2023, 2022, 2020])

In [313]:
import numpy as np

final_data["sin_year"] = np.sin(2 * np.pi * final_data["year"] / final_data["year"].max())
final_data["cos_year"] = np.cos(2 * np.pi * final_data["year"] / final_data["year"].max())

In [314]:
final_data.tail()

,warehouse,total_orders,sales,sell_price_main,availability,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,...,weight,min_discount,max_discount,year,month,day,weekday,day_of_week,sin_year,cos_year
4007414,4,9988.0,1.929869,34.06,1.0,2422,1239,1,25,65,...,2.262646,0.0,0.0,2023,6,21,2,6,-0.003104,0.999995
4007415,4,8518.0,1.929869,34.06,1.0,2422,1239,1,25,65,...,2.262646,0.0,0.0,2023,6,24,5,2,-0.003104,0.999995
4007416,4,10424.0,1.929869,34.06,1.0,2422,1239,1,25,65,...,2.262646,0.0,0.0,2023,6,23,4,0,-0.003104,0.999995
4007417,4,10342.0,1.929869,34.06,1.0,2422,1239,1,25,65,...,2.262646,0.0,0.0,2023,6,22,3,4,-0.003104,0.999995
4007418,4,9613.0,1.929869,34.06,1.0,2422,1239,1,25,65,...,2.262646,0.0,0.0,2023,6,20,1,5,-0.003104,0.999995


In [315]:
final_data.drop(columns=['product_unique_id','month','day','weekday','day_of_week'],inplace=True)

In [316]:
"final_data.to_csv('Cleaned_data.csv',index=True)"

"final_data.to_csv('Cleaned_data.csv',index=True)"

In [317]:
final_data.drop(columns=['year','availability'],inplace=True)

In [318]:
final_data.head()

,warehouse,total_orders,sales,sell_price_main,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,weight,min_discount,max_discount,sin_year,cos_year
0,1,6436.0,1.929869,646.26,960,0,2,52,0,0,31,0,1.925596,0.0,0.15312,-2.449294e-16,1.000000
1,1,4663.0,1.929869,455.96,960,0,2,52,0,0,31,0,1.925596,0.0,0.15025,-9.312887e-03,0.999957
2,1,6507.0,1.929869,455.96,960,0,2,52,0,0,31,0,1.925596,0.0,0.15025,-9.312887e-03,0.999957
3,1,5463.0,1.929869,646.26,960,0,2,52,0,0,31,0,1.925596,0.0,0.20024,-3.104336e-03,0.999995
4,1,5997.0,1.929869,486.41,960,0,2,52,0,0,31,0,1.925596,0.0,0.15649,-6.208641e-03,0.999981


In [319]:
"""from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

final_data['sell_price_main'] = scaler.fit_transform(final_data[['sell_price_main']])
"""

"from sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\n\nfinal_data['sell_price_main'] = scaler.fit_transform(final_data[['sell_price_main']])\n"

Splitting the values in X and Y

In [320]:
X = final_data.drop(columns='sales')
Y = final_data['sales']

Importing the models and other libraries

In [321]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score


In [322]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [323]:
#Xgboost model

model = RandomForestRegressor()

model.fit(X_train,Y_train)

RandomForestRegressor()

In [324]:
Y_pred = model.predict(X_test)
Y_pred

array([1.92986903, 1.92986903, 1.92986903, ..., 1.92986903, 1.92986903,
       1.92986903])

In [325]:
"""from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Y_pred = scaler.fit_transform(Y_pred.reshape(-1, 1))
Y_pred"""

'from sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\nY_pred = scaler.fit_transform(Y_pred.reshape(-1, 1))\nY_pred'

In [326]:
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
print(f'Root Mean Squared Error (RMSE): {rmse}')

Root Mean Squared Error (RMSE): 3.4607301777864974e-11


Kaggle_test_data

In [327]:
Test_data = pd.read_csv(r'Datasets/sales_test.csv')

In [328]:
Test_data = Test_data.merge(inventory[['unique_id', 'product_unique_id', 'name',
                                'L1_category_name_en', 'L2_category_name_en',
                                'L3_category_name_en', 'L4_category_name_en']],
                on='unique_id', how='left')

In [329]:
Test_data = Test_data.merge(calendar[['date', 'warehouse', 'holiday', 'holiday_name',
                                'shops_closed', 'winter_school_holidays', 'school_holidays']],
                on=['date', 'warehouse'], how='left')


In [330]:
Test_data = Test_data.merge(test_weight[['unique_id','weight']],on='unique_id', how='left')

In [331]:
Test_data.head()

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,...,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,winter_school_holidays,school_holidays,weight
0,1226,2024-06-03,Brno_1,8679.0,13.13,0.00000,0.0,0.0,0.0,0.0,...,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,0,NaN,0,0,0,0.811295
1,1226,2024-06-11,Brno_1,8795.0,13.13,0.15873,0.0,0.0,0.0,0.0,...,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,0,NaN,0,0,0,0.811295
2,1226,2024-06-13,Brno_1,10009.0,13.13,0.15873,0.0,0.0,0.0,0.0,...,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,0,NaN,0,0,0,0.811295
3,1226,2024-06-15,Brno_1,8482.0,13.13,0.15873,0.0,0.0,0.0,0.0,...,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,0,NaN,0,0,0,0.811295
4,1226,2024-06-09,Brno_1,8195.0,13.13,0.00000,0.0,0.0,0.0,0.0,...,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,0,NaN,0,0,0,0.811295


In [332]:
Test_data.isnull().sum()

unique_id                     0
date                          0
warehouse                     0
total_orders                  0
sell_price_main               0
type_0_discount               0
type_1_discount               0
type_2_discount               0
type_3_discount               0
type_4_discount               0
type_5_discount               0
type_6_discount               0
product_unique_id             0
name                          0
L1_category_name_en           0
L2_category_name_en           0
L3_category_name_en           0
L4_category_name_en           0
holiday                       0
holiday_name              46544
shops_closed                  0
winter_school_holidays        0
school_holidays               0
weight                        0
dtype: int64

In [333]:
#Handling the missing values of total_orders
Test_data['holiday_name'] = Test_data['holiday_name'].fillna('No Holiday')


In [334]:
#Converting the types discount into the min and max discount
Test_data['min_discount'] = Test_data[['type_0_discount', 'type_1_discount', 'type_2_discount',
                                     'type_3_discount', 'type_4_discount', 'type_5_discount',
                                     'type_6_discount']].min(axis=1)
Test_data['max_discount'] = Test_data[['type_0_discount', 'type_1_discount', 'type_2_discount',
                                  'type_3_discount', 'type_4_discount', 'type_5_discount',
                                  'type_6_discount']].max(axis=1)


In [335]:
Test_data.drop(columns=['type_0_discount', 'type_1_discount', 'type_2_discount',
                                  'type_3_discount', 'type_4_discount', 'type_5_discount',
                                  'type_6_discount'],inplace=True)

In [336]:
Test_data["name"].nunique()

1671

In [337]:
Test_data

,unique_id,date,warehouse,total_orders,sell_price_main,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,winter_school_holidays,school_holidays,weight,min_discount,max_discount
0,1226,2024-06-03,Brno_1,8679.0,13.13,627,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,0,No Holiday,0,0,0,0.811295,0.0,0.00000
1,1226,2024-06-11,Brno_1,8795.0,13.13,627,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,0,No Holiday,0,0,0,0.811295,0.0,0.15873
2,1226,2024-06-13,Brno_1,10009.0,13.13,627,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,0,No Holiday,0,0,0,0.811295,0.0,0.15873
3,1226,2024-06-15,Brno_1,8482.0,13.13,627,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,0,No Holiday,0,0,0,0.811295,0.0,0.15873
4,1226,2024-06-09,Brno_1,8195.0,13.13,627,Croissant_9,Bakery,Bakery_L2_14,Bakery_L3_37,Bakery_L4_1,0,No Holiday,0,0,0,0.811295,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47016,4572,2024-06-03,Munich_1,5254.0,2.09,2245,Apple_123,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_31,Fruit and vegetable_L4_1,0,No Holiday,0,0,0,3.682678,0.0,0.00000
47017,3735,2024-06-04,Prague_1,9698.0,11.00,1833,Kiwi_18,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_39,Fruit and vegetable_L4_52,0,No Holiday,0,0,0,0.754215,0.0,0.00000
47018,3735,2024-06-03,Prague_1,10256.0,11.00,1833,Kiwi_18,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_39,Fruit and vegetable_L4_52,0,No Holiday,0,0,0,0.754215,0.0,0.00000
47019,2129,2024-06-03,Brno_1,8679.0,37.75,1074,Grape_15,Fruit and vegetable,Fruit and vegetable_L2_1,Fruit and vegetable_L3_12,Fruit and vegetable_L4_1,0,No Holiday,0,0,0,2.274807,0.0,0.00000


In [338]:
#date handling
Test_data['date'] = pd.to_datetime(Test_data['date'])

In [339]:
Test_data['year'] = Test_data['date'].dt.year


In [340]:
Test_data.drop(columns='date',inplace=True)

In [341]:
Test_data.drop(columns=['unique_id','winter_school_holidays','school_holidays'],inplace=True)

In [342]:
Test_data.shape

(47021, 16)

In [343]:
Test_data['warehouse'] = Test_data['warehouse'].str.replace('_', '', regex=False)
Test_data['name'] = Test_data['name'].str.replace('_', '', regex=False)
Test_data['L1_category_name_en'] = Test_data['L1_category_name_en'].str.replace('_', '', regex=False)
Test_data['L2_category_name_en'] = Test_data['L2_category_name_en'].str.replace('_', '', regex=False)
Test_data['L3_category_name_en'] = Test_data['L3_category_name_en'].str.replace('_', '', regex=False)
Test_data['L4_category_name_en'] = Test_data['L4_category_name_en'].str.replace('_', '', regex=False)
Test_data['holiday_name'] = Test_data['holiday_name'].str.replace('_', '', regex=False)


In [344]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Apply the label encoder to the 'warehouse' column
Test_data['warehouse'] = label_encoder.fit_transform(Test_data['warehouse'])


In [345]:
Test_data.shape

(47021, 16)

In [346]:
Test_data['name'] = Test_data['name'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
Test_data['L1_category_name_en'] = Test_data['L1_category_name_en'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
Test_data['L2_category_name_en'] = Test_data['L2_category_name_en'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
Test_data['L3_category_name_en'] = Test_data['L3_category_name_en'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
Test_data['L4_category_name_en'] = Test_data['L4_category_name_en'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
Test_data['holiday_name'] = Test_data['holiday_name'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()


In [347]:
#performing encoding
Test_data['name'] = label_encoder.fit_transform(Test_data['name'])
Test_data['L1_category_name_en'] = label_encoder.fit_transform(Test_data['L1_category_name_en'])
Test_data['L2_category_name_en'] = label_encoder.fit_transform(Test_data['L2_category_name_en'])
Test_data['L3_category_name_en'] = label_encoder.fit_transform(Test_data['L3_category_name_en'])
Test_data['L4_category_name_en'] = label_encoder.fit_transform(Test_data['L4_category_name_en'])
Test_data['holiday_name'] = label_encoder.fit_transform(Test_data['holiday_name'])


In [348]:
import numpy as np

Test_data["sin_year"] = np.sin(2 * np.pi * Test_data["year"] / Test_data["year"].max())
Test_data["cos_year"] = np.cos(2 * np.pi * Test_data["year"] / Test_data["year"].max())

In [349]:
Test_data.drop(columns='year',inplace=True)

In [350]:
Test_data.head()

,warehouse,total_orders,sell_price_main,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,weight,min_discount,max_discount,sin_year,cos_year
0,0,8679.0,13.13,627,666,0,1,32,0,0,1,0,0.811295,0.0,0.00000,-2.449294e-16,1.0
1,0,8795.0,13.13,627,666,0,1,32,0,0,1,0,0.811295,0.0,0.15873,-2.449294e-16,1.0
2,0,10009.0,13.13,627,666,0,1,32,0,0,1,0,0.811295,0.0,0.15873,-2.449294e-16,1.0
3,0,8482.0,13.13,627,666,0,1,32,0,0,1,0,0.811295,0.0,0.15873,-2.449294e-16,1.0
4,0,8195.0,13.13,627,666,0,1,32,0,0,1,0,0.811295,0.0,0.00000,-2.449294e-16,1.0


In [351]:
final_data.head()

,warehouse,total_orders,sales,sell_price_main,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,weight,min_discount,max_discount,sin_year,cos_year
0,1,6436.0,1.929869,646.26,960,0,2,52,0,0,31,0,1.925596,0.0,0.15312,-2.449294e-16,1.000000
1,1,4663.0,1.929869,455.96,960,0,2,52,0,0,31,0,1.925596,0.0,0.15025,-9.312887e-03,0.999957
2,1,6507.0,1.929869,455.96,960,0,2,52,0,0,31,0,1.925596,0.0,0.15025,-9.312887e-03,0.999957
3,1,5463.0,1.929869,646.26,960,0,2,52,0,0,31,0,1.925596,0.0,0.20024,-3.104336e-03,0.999995
4,1,5997.0,1.929869,486.41,960,0,2,52,0,0,31,0,1.925596,0.0,0.15649,-6.208641e-03,0.999981


In [352]:
Test_data.drop(columns='product_unique_id',inplace=True)

In [353]:
Test_data.head()

,warehouse,total_orders,sell_price_main,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,holiday,holiday_name,shops_closed,weight,min_discount,max_discount,sin_year,cos_year
0,0,8679.0,13.13,666,0,1,32,0,0,1,0,0.811295,0.0,0.00000,-2.449294e-16,1.0
1,0,8795.0,13.13,666,0,1,32,0,0,1,0,0.811295,0.0,0.15873,-2.449294e-16,1.0
2,0,10009.0,13.13,666,0,1,32,0,0,1,0,0.811295,0.0,0.15873,-2.449294e-16,1.0
3,0,8482.0,13.13,666,0,1,32,0,0,1,0,0.811295,0.0,0.15873,-2.449294e-16,1.0
4,0,8195.0,13.13,666,0,1,32,0,0,1,0,0.811295,0.0,0.00000,-2.449294e-16,1.0


In [354]:
"Test_data['sell_price_main'] = scaler.fit_transform(Test_data[['sell_price_main']])"

"Test_data['sell_price_main'] = scaler.fit_transform(Test_data[['sell_price_main']])"

In [355]:
Test_Y_pred = model.predict(Test_data)

In [356]:
Test_Y_pred

array([1.92986903, 1.92986903, 1.92986903, ..., 1.92986903, 1.92986903,
       1.92986903])

In [357]:
sample = pd.read_csv(r'Datasets/sales_test.csv')

In [358]:
solution = pd.read_csv(r'Datasets/Solution.csv')

In [359]:
solution

,id,sales_hat
0,1226_2024-06-03,0
1,1226_2024-06-11,0
2,1226_2024-06-13,0
3,1226_2024-06-15,0
4,1226_2024-06-09,0
...,...,...
47016,4572_2024-06-03,0
47017,3735_2024-06-04,0
47018,3735_2024-06-03,0
47019,2129_2024-06-03,0


In [360]:
sample['id'] = sample['unique_id'].astype(str) + '_' + sample['date']

Final_prediction = pd.DataFrame({
    'id': sample['id'][:len(Test_Y_pred)],  # Align lengths
    'sales_hat': Test_Y_pred
})


In [361]:
Final_prediction

,id,sales_hat
0,1226_2024-06-03,1.929869
1,1226_2024-06-11,1.929869
2,1226_2024-06-13,1.929869
3,1226_2024-06-15,1.929869
4,1226_2024-06-09,1.929869
...,...,...
47016,4572_2024-06-03,1.929869
47017,3735_2024-06-04,1.929869
47018,3735_2024-06-03,1.929869
47019,2129_2024-06-03,1.929869


In [362]:
Final_prediction.to_csv("Datasets/Final_Submission.csv", index=False)

Deploying the model


In [363]:
import pickle

with open('Models/model.pkl', 'wb') as file:
    pickle.dump(model, file)
